<a href="https://colab.research.google.com/github/wily037/Ameyo-data-minds/blob/main/Tai_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:

code = """
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
import io

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/analyze/")
async def analyze_csv(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        df = pd.read_csv(io.StringIO(contents.decode("utf-8")))

        summary = {
            "columns": df.columns.tolist(),
            "shape": df.shape,
            "missing_values": df.isnull().sum().to_dict(),
            "mean": df.mean(numeric_only=True).to_dict(),
            "median": df.median(numeric_only=True).to_dict(),
            "head": df.head().to_dict(orient="records")
        }

        return {"status": "success", "summary": summary}

    except Exception as e:
        return {"status": "error", "message": str(e)}
"""

with open("main.py", "w") as f:
    f.write(code)
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
import io

app = FastAPI()

# Enable CORS (so frontend can call the API later)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Change to specific domain in production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/analyze/")
async def analyze_csv(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        df = pd.read_csv(io.StringIO(contents.decode("utf-8")))

        # Summary statistics
        summary = {
            "columns": df.columns.tolist(),
            "shape": df.shape,
            "missing_values": df.isnull().sum().to_dict(),
            "mean": df.mean(numeric_only=True).to_dict(),
            "median": df.median(numeric_only=True).to_dict(),
            "head": df.head().to_dict(orient="records")
        }

        return {"status": "success", "summary": summary}

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [15]:
!pip install fastapi uvicorn nest-asyncio pyngrok pandas

In [ ]:

from pyngrok import ngrok

# Paste your auth token below between the quotes
ngrok.set_auth_token("30KcWzC7n4I7sWgVgLAQLseeJGB_5QwBLdsMuQTF2jFkvmLbX")

In [16]:

from fastapi import File, UploadFile
import pandas as pd
import io

@app.post("/upload_csv/")
async def upload_csv(file: UploadFile = File(...)):
    contents = await file.read()
    df = pd.read_csv(io.StringIO(contents.decode("utf-8")))
    return {
        "filename": file.filename,
        "columns": df.columns.tolist(),
        "rows": len(df)
    }

In [ ]:

from pyngrok import ngrok
ngrok.kill()

In [ ]:

# Install FastAPI, Uvicorn, Pandas, Ngrok (run this in Colab)
!pip install fastapi[all] pyngrok pandas openpyxl --quiet

# --- main.py (Put this in a new cell in Colab) ---
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
from io import BytesIO

app = FastAPI()

# Allow CORS (Optional if you'll use frontend later)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/upload/")
async def upload_file(file: UploadFile = File(...)):
    contents = await file.read()
    try:
        if file.filename.endswith(".csv"):
            df = pd.read_csv(BytesIO(contents))
        elif file.filename.endswith(".xlsx"):
            df = pd.read_excel(BytesIO(contents))
        else:
            return JSONResponse(content={"error": "Unsupported file format. Upload .csv or .xlsx"}, status_code=400)
    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)

    return {
        "message": "File uploaded successfully!",
        "columns": df.columns.tolist(),
        "rows": df.shape[0]
    }

# Run the server
from pyngrok import ngrok
import uvicorn
import nest_asyncio
nest_asyncio.apply()

public_url = ngrok.connect(8000)
print(f"🔗 Your Public API: {public_url}")
uvicorn.run(app, port=8000)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.1/453.1 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.6/950.6 kB 57.3 MB/s eta 0:00:00


INFO:     Started server process [959]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


🔗 Your Public API: NgrokTunnel: "https://5b4bb5dc3f9e.ngrok-free.app" -> "http://localhost:8000"


In [ ]:

from fastapi import File, UploadFile
from fastapi.responses import JSONResponse
import pandas as pd
import io

@app.post("/upload/")
async def upload_file(file: UploadFile = File(...)):
    contents = await file.read()
    try:
        df = pd.read_csv(io.BytesIO(contents))
        preview = {
            "columns": df.columns.tolist(),
            "shape": df.shape,
            "head": df.head().to_dict()
        }
        return JSONResponse(content=preview)
    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=400)

In [ ]:
import nest_asyncio
import threading
from pyngrok import ngrok
import uvicorn

# Allow async code in Colab
nest_asyncio.apply()

# Set up the public URL
public_url = ngrok.connect(8000)
print(f"🔗 Your Public API: {public_url}/docs")

# Function to run server
def run():
    uvicorn.run("main:app", host="0.0.0.0", port=8000)

# Start server in background
thread = threading.Thread(target=run)
thread.start()

🔗 Your Public API: NgrokTunnel: "https://228467d203b1.ngrok-free.app" -> "http://localhost:8000"/docs


Exception in thread Thread-10 (run):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run


In [ ]:

# Install requirements (only needed once per session)
!pip install fastapi uvicorn pyngrok python-multipart openpyxl --quiet

# Save main FastAPI app
with open("main.py", "w") as f:
    f.write("""
from fastapi import FastAPI, File, UploadFile, HTTPException
import pandas as pd
from io import BytesIO

app = FastAPI()

@app.get("/")
def root():
    return {"message": "Hello, this is your API!"}

@app.post("/upload/")
async def upload_file(file: UploadFile = File(...)):
    try:
        if file.filename.endswith(".csv"):
            df = pd.read_csv(BytesIO(await file.read()))
        elif file.filename.endswith(".xlsx"):
            df = pd.read_excel(BytesIO(await file.read()))
        else:
            raise HTTPException(status_code=400, detail="Only CSV or Excel files are supported")
        return {
            "columns": df.columns.tolist(),
            "rows": df.head(5).to_dict(orient="records")
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
""")

# Setup Ngrok tunnel
from pyngrok import ngrok
import time
try:
    ngrok.kill()  # Close previous tunnels if any
except:
    pass
public_url = ngrok.connect(8000)
print(f"🔗 Your Public API: {public_url}/docs")

# Start FastAPI using uvicorn in background
import nest_asyncio
import uvicorn
nest_asyncio.apply()

uvicorn.run("main:app", host="0.0.0.0", port=8000)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-4' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

🔗 Your Public API: NgrokTunnel: "https://91b3dce645b0.ngrok-free.app" -> "http://localhost:8000"/docs


INFO:     Started server process [959]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
